In [1]:
import os
import torch
import torch_xla.core.xla_model as xm
from transformers import AutoTokenizer, AutoModelForCausalLM

print("Étape 0: Initialisation.")

os.environ["XLA_USE_BF16"] = "1"
os.environ["TORCH_XLA_DEBUG"] = "0"
MODEL_ID = "utter-project/EuroLLM-1.7B-Instruct"

# --- ÉTAPE 1 ---
print("\nÉtape 1: Connexion au TPU...")
device = xm.xla_device()
print(f"Device TPU trouvé: {device}")

# --- ÉTAPE 2 ---
print("\nÉtape 2: Téléchargement du tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
print("Tokenizer téléchargé.")

# --- ÉTAPE 3 ---
print("\nÉtape 3: Téléchargement du modèle...")
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype=torch.bfloat16)
model.config.use_cache = False  # XLA incompatible
model.to(device)
xm.mark_step()
print("Modèle prêt sur TPU.")

# --- ÉTAPE 4 ---
prompt = "Translate this text into french : 'to be or not to be that is the question'"
print("\nÉtape 4: Tokenisation du prompt...")
inputs = tokenizer(prompt, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]
print("Prompt tokenizé.")

# --- ÉTAPE 5 ---
print("\nÉtape 5: Warm-up TPU (compilation initiale courte)...")
with torch.no_grad():
    _ = model(input_ids)
xm.mark_step()
print("Warm-up terminé.")

# --- ÉTAPE 6 ---
print("\nÉtape 6: Génération manuelle token par token (optimisée pour TPU)...")

max_new_tokens = 64
generated_ids = input_ids
model.eval()

with torch.no_grad():
    for i in range(max_new_tokens):
        # Forward
        outputs = model(generated_ids)
        next_token_logits = outputs.logits[:, -1, :]

        # Prendre le token le plus probable (greedy)
        next_token = torch.argmax(next_token_logits, dim=-1, keepdim=True)

        # Ajouter le nouveau token à la séquence
        generated_ids = torch.cat([generated_ids, next_token], dim=-1)

        # Synchronisation TPU
        xm.mark_step()

        # Vérifie si fin de séquence atteinte
        if next_token.item() == tokenizer.eos_token_id:
            print(f"\nArrêt précoce à {i+1} tokens (EOS atteint).")
            break

        # Affiche un aperçu tous les 10 tokens
        if (i + 1) % 10 == 0:
            partial = tokenizer.decode(generated_ids[0].cpu(), skip_special_tokens=True)
            print(f"→ {i+1} tokens générés...")
            print(partial)
            print("-" * 50)

print("\nGénération terminée.")

# --- ÉTAPE 7 ---
print("\nÉtape 7: Décodage final...")
generated_text = tokenizer.decode(generated_ids[0].cpu(), skip_special_tokens=True)

print("\n--- RÉSULTAT FINAL ---")
print(generated_text)

/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


Étape 0: Initialisation.

Étape 1: Connexion au TPU...
Device TPU trouvé: xla:0

Étape 2: Téléchargement du tokenizer...


/tmp/ipython-input-3215314520.py:14: DeprecationWarning: Use torch_xla.device instead
  device = xm.xla_device()


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

/tmp/ipython-input-3215314520.py:29: DeprecationWarning: Use torch_xla.sync instead
  xm.mark_step()
/tmp/ipython-input-3215314520.py:43: DeprecationWarning: Use torch_xla.sync instead
  xm.mark_step()


Modèle prêt sur TPU.

Étape 4: Tokenisation du prompt...
Prompt tokenizé.

Étape 5: Warm-up TPU (compilation initiale courte)...
Warm-up terminé.

Étape 6: Génération manuelle token par token (optimisée pour TPU)...


/tmp/ipython-input-3215314520.py:66: DeprecationWarning: Use torch_xla.sync instead
  xm.mark_step()


→ 10 tokens générés...
Translate this text into french : 'to be or not to be that is the question'
« être ou ne pas être,
--------------------------------------------------
→ 20 tokens générés...
Translate this text into french : 'to be or not to be that is the question'
« être ou ne pas être, telle est la question »

«
--------------------------------------------------
→ 30 tokens générés...
Translate this text into french : 'to be or not to be that is the question'
« être ou ne pas être, telle est la question »

« être ou ne pas être, telle est
--------------------------------------------------
→ 40 tokens générés...
Translate this text into french : 'to be or not to be that is the question'
« être ou ne pas être, telle est la question »

« être ou ne pas être, telle est la question »

« 
--------------------------------------------------
→ 50 tokens générés...
Translate this text into french : 'to be or not to be that is the question'
« être ou ne pas être, telle est la question »

